<a href="https://colab.research.google.com/github/OlehVakulchyk/DataScience/blob/main/Hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

****
# HomeWork 5
За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою.

Для використання алгоритму SVM та Random Forest створимо датасет з часовими ознаками файлів з датасету (min, max, mean, std)

In [42]:
import pandas as pd
import numpy as np
import zipfile
import os

from scipy.stats import entropy, skew
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score


In [43]:
zip_files = '/content/drive/MyDrive/Colab Notebooks/homework.zip'

with zipfile.ZipFile(zip_files, 'r') as zip_ref:
    zip_ref.extractall('unzip_folder')


In [44]:
def load_data_frame(folder_path):

    data = pd.DataFrame()
    type_activity = os.path.basename(folder_path)
    for filename in os.listdir(folder_path):

      if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path, filename))
        data_d = pd.DataFrame(
        {
        'max_x': df['accelerometer_X'].max(),
        'min_x': df['accelerometer_X'].min(),
        'mean_x': df['accelerometer_X'].mean(),
        'std_x': df['accelerometer_X'].std(),

        'max_y': df['accelerometer_Y'].max(),
        'min_y': df['accelerometer_Y'].min(),
        'mean_y': df['accelerometer_Y'].mean(),
        'std_y': df['accelerometer_Y'].std(),

        'max_z': df['accelerometer_Z'].max(),
        'min_z': df['accelerometer_Z'].min(),
        'mean_z': df['accelerometer_Z'].mean(),
        'std_z': df['accelerometer_Z'].std(),
        'type_activity': type_activity,
        }, index=[0]
        )
        data = pd.concat([data, data_d]).reset_index(drop=True)

    return data

In [45]:
walk_path = 'unzip_folder/data/walking'
stairs_path = 'unzip_folder/data/stairs'
run_path = 'unzip_folder/data/running'
idle_path = 'unzip_folder/data/idle'


walk_df = load_data_frame(walk_path)
stairs_df = load_data_frame(stairs_path)
run_df = load_data_frame(run_path)
idle_df = load_data_frame(idle_path)



df = pd.concat([walk_df, stairs_df, run_df, idle_df]).reset_index(drop=True)


df.head()


,max_x,min_x,mean_x,std_x,max_y,min_y,mean_y,std_y,max_z,min_z,mean_z,std_z,type_activity
0,11.894394,-12.277467,0.237824,5.549729,6.435614,-23.673866,-11.556971,6.137406,5.612009,-14.245501,-1.720952,4.821954,walking
1,2.245761,-15.026010,-4.941313,4.601738,8.834604,-28.773516,-10.466013,7.700976,13.254301,-33.949780,-2.397553,9.136955,walking
2,0.464475,-25.665842,-7.151480,6.359630,0.349553,-22.855050,-11.929350,5.509035,12.775460,-25.838224,-3.386199,7.963452,walking
3,3.701436,-15.519216,-4.425442,4.181275,13.359646,-17.290924,-8.902759,6.898776,19.589357,-19.919758,-1.777615,7.726321,walking
4,5.990293,-7.958326,-0.627281,2.949334,-0.220267,-19.632454,-8.464620,5.216559,8.149863,-13.163321,-0.405737,4.968402,walking


Визначимо тестову вибірку та знайдемо найкращу модель SVM за допомогою GridSearchCV

In [46]:
X = df.drop('type_activity', axis=1)
y = df['type_activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, train_size=0.7, random_state=42)

svm_classifier = SVC()
svm_param_grid = {'C': [0.1, 1, 10, 100],
                  'gamma': [1, 0.1, 0.01, 0.001],
                  'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}
svm_grid_search = GridSearchCV(svm_classifier, svm_param_grid, cv=5, scoring='accuracy')
svm_grid_search.fit(X_train, y_train)

best_svm_params = svm_grid_search.best_params_
best_svm_model = svm_grid_search.best_estimator_

print(best_svm_params)
print(best_svm_model)

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=10, gamma=0.01)


In [47]:
print(best_svm_params)
print(best_svm_model)

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=10, gamma=0.01)


Знайдемо найкращу модель Random Forest за допомогою GridSearchCV

In [48]:
rf_classifier = RandomForestClassifier()
rf_param_grid = {'n_estimators': [50, 100, 200],
                 'max_depth': [None, 10, 20, 30],
                 'min_samples_split': [2, 5, 10]}

rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=5, scoring='accuracy')
rf_grid_search.fit(X_train, y_train)

best_rf_params = rf_grid_search.best_params_
best_rf_model = rf_grid_search.best_estimator_


print(best_rf_params)


{'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}


Знайдемо показники точності та F1 для алгоритмів SVM та Random Forest

In [49]:
svm_preds = best_svm_model.predict(X_test)
rf_preds = best_rf_model.predict(X_test)

svm_accuracy = accuracy_score(y_test, svm_preds)
rf_accuracy = accuracy_score(y_test, rf_preds)

svm_f1 = f1_score(y_test, svm_preds, average='weighted')
rf_f1 = f1_score(y_test, rf_preds, average='weighted')

print(f'SVM accuracy: {svm_accuracy}')
print(f'Random Forest accuracy: {rf_accuracy}')
print(f'SVM F1: {svm_f1}')
print(f'Random Forest F1: {rf_f1}')

SVM accuracy: 0.9984532095901005
Random Forest accuracy: 0.9992266047950503
SVM F1: 0.99847270972561
Random Forest F1: 0.9992214943486407


**Висновок:** В цілому, обидві моделі досягли високої точності та F1-міри, що свідчить про їхню хорошу роботу у класифікації діяльностей на основі даних акселерометра. Але модель Random Forest має невелику перевагу над моделлю SVM у цьому конкретному наборі даних.

In [50]:
# Random Forest
param_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

random_rf = RandomizedSearchCV(RandomForestClassifier(), param_rf, n_iter=10, cv=5)
random_rf.fit(X_train, y_train)

print("Best parameters for Random Forest: ", random_rf.best_params_)



Best parameters for Random Forest:  {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': None}


In [51]:
# SVM
param_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

random_svm = RandomizedSearchCV(SVC(), param_svm, n_iter=10, cv=5)
random_svm.fit(X_train, y_train)

print("Best parameters for SVM: ", random_svm.best_params_)

Best parameters for SVM:  {'kernel': 'rbf', 'gamma': 0.01, 'C': 10}


RandomizedSearchCV працює дуже схоже до GridSearchCV, але замість того, щоб пробувати всі можливі комбінації параметрів, він випадковим чином вибирає певну кількість комбінацій. Це може бути набагато швидше, особливо якщо у вас багато параметрів для налаштування. Але при декількох виконаннях з однаковими параметрами результати можуть не співпадати завдяки випадковості комбінацій.

По часу виконання в мене для SVM:

 RandomizedSearchCV - 30-50 cекунд

 GridSearchCV - 4 хвилини